<a href="https://colab.research.google.com/github/7ft10/JiraExporter/blob/main/DataFlows - Server/Jira_Tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#! Library Import

import os 
import importlib.util

if importlib.util.find_spec("pandas") is None:	
	os.system("pip install pandas")

if importlib.util.find_spec("dotenv") is None:	
	os.system("pip install --quiet openai python-dotenv")

In [2]:
#! pandas Config

import pandas as pd
import requests
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

pd.options.mode.use_inf_as_na = True
#pd.set_option("display.max_rows", 10)
#pd.set_option("display.expand_frame_repr", True)
#pd.set_option('display.width', 1000)
#pd.options.display.max_seq_items = 200000
#pd.options.display.max_rows = 10

In [3]:
#! Parameters

import os 
import dotenv
import importlib.util

if importlib.util.find_spec("google.colab") is not None: ## if using google colab
    if not os.path.exists('.env'):
        from google.colab import files
        uploaded = files.upload()
        file_name = list(uploaded.keys())[0]
        try:
            os.rename(file_name, '.env')
        except:
            pass

dotenv.load_dotenv('.env')

Host = os.getenv('SECRETS_HOST')
Username = os.getenv('SECRETS_USERNAME')
Password = os.getenv('SECRETS_PASSWORD')
ValidProjectCategories = ["'Portfolio'"]

if Host is None:
    raise Exception("Secrets not found")

display("Host: " + Host)
display("ValidProjectCategories: " + ','.join(ValidProjectCategories))

'Host: https://jira.budgetdirect.com.au/'

"ValidProjectCategories: 'Portfolio'"

In [4]:
#! Functions

import base64
import pandas as pd
import re 
import requests
import warnings
from functools import reduce
    
def _ExpandColumn(self:pd.DataFrame, colName:str, columnsToExpand = [], prefix:str = "Prefix", sentenceCase:bool = True) -> pd.DataFrame:
    if (prefix == "Prefix"):
        prefix = colName + " "
        with warnings.catch_warnings():
          warnings.simplefilter(action='ignore', category=FutureWarning)
          expandedCols = self[colName].apply(lambda x: pd.Series(x).add_prefix(prefix))
        columnsToExpand = [prefix + c for c in columnsToExpand]
    else:
        expandedCols = self[colName].apply(lambda x: pd.Series(x))
    
    if len(columnsToExpand) > 0:        
        expandedCols = expandedCols[columnsToExpand]
    
    if sentenceCase:
        expandedCols.columns = [fnSentenceCase(c) for c in expandedCols.columns] 

    return pd.concat([self.drop(colName, axis=1), expandedCols], axis=1)

pd.DataFrame.expand = _ExpandColumn

def fnSentenceCase(s):
    s = (' '.join(dict.fromkeys(s.split())))  # remove duplicate words
    s = s.replace("0", "") # remove "0" 
    s = s.strip()
    return ' '.join([x.capitalize() for x in re.sub(r"([A-Z])", r" \1", s).split()]) # sentence case

def _SentenceCaseColumns(self:pd.DataFrame) -> pd.DataFrame: 
    self.columns = [fnSentenceCase(c) for c in self.columns] 
    return self

pd.DataFrame.sentence_case_columns = _SentenceCaseColumns

def fnGetDefaultHeaders():
    return {
        "content-type": "application/json",
        "authorization": "Basic " + base64.b64encode((Username + ":" + Password).encode()).decode(),
        "retry-after": "120"
    }

def fnSearch(jql, fields = None, expand = None):
    def ApiCall(startAt) :
        url = "/rest/api/latest/search"
        headers = fnGetDefaultHeaders()
        defaultContents = {
            "startAt": startAt,
            "maxResults": "2",
            "jql": jql
        }
        if fields is not None:
            defaultContents["fields"] = fields.tolist()
        if expand is not None and expand != "":
            defaultContents["expand"] = expand        
        response = requests.post(Host + url, headers = headers, json = defaultContents, verify=False)
        return response.json()
    values = fnAPI(ApiCall)
    if len(values.index) > 1:
        return values
    else:
        None

def fnGetIssueTypeFields(IssueTypes) -> pd.DataFrame:
    def ApiCall(startAt) :
        url = "rest/api/latest/issue/createmeta"
        headers = fnGetDefaultHeaders()
        params = {
            "expand": "projects.issuetypes.fields",
            "projectKeys": ','.join(fnGetValidProjectKeys()["key"].values),
            "issuetypeNames": ','.join(IssueTypes).replace("'", "")
        }
        response = requests.get(Host + url, headers = headers, params = params, verify=False)
        return response.json()
    
    df = fnAPI(ApiCall)
    df = df.drop(["expand"], axis=1)
    df = df.explode("projects")
    df = df.expand("projects", [], None, False)    
    try:
        df = df[["issuetypes"]]
    except: 
        raise Exception("No issue metadata - check the valid project categories are correct")
    df = df.explode("issuetypes")
    df = df.expand("issuetypes", [], None, False)
    df = df[["fields"]]
    df = df.expand("fields", [], None, False)
    df = df.loc[:,~df.columns.duplicated()]

    values = []
    for x in df.columns:        
        try:            
            valid:pd.DataFrame = pd.DataFrame( df[~df[x].isnull()] )[[x]].iloc[0].get(0)
            values.append({
                "fieldId": valid['key'] if "key" in valid else valid["fieldId"],
                "name": valid['name'],
                "schema_type": valid['schema']['type'],
                "required": valid['required']
            })
        except:
            #display(x)
            pass
    values.append({ "fieldId": 'status', "name": 'Status', "schema_type": 'string', "required": True })
    values.append({ "fieldId": 'created', "name": 'Created', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'updated', "name": 'Updated', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'resolution', "name": 'Resolution', "schema_type": 'string', "required": True })
    values.append({ "fieldId": 'resolutiondate', "name": 'Resolution Date', "schema_type": 'date', "required": False })
    values.append({ "fieldId": 'lastViewed', "name": 'Last Viewed', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'id', "name": 'Id', "schema_type": 'number', "required": True })
    values.append({ "fieldId": 'key', "name": 'Key', "schema_type": 'string', "required": True })
    df = pd.DataFrame(values)
    df = df.drop_duplicates().sort_values("fieldId")
    return df 

def fnGetValidProjectKeys() -> pd.DataFrame:
    def ApiCall(startAt) :
        url = "/rest/api/latest/project"
        headers = fnGetDefaultHeaders()
        params = { }
        response = requests.get(Host + url, headers = headers, params = params, verify=False)
        return response.json()
    df = fnAPI(ApiCall)
    df = df.expand("projectCategory")
    if 'ValidProjectCategories' in globals() and len(ValidProjectCategories) > 0:
        df = df.loc[df['Project Category Name'].isin(ValidProjectCategories) | ("'" + df['Project Category Name'] + "'").isin(ValidProjectCategories)]
    return df[["key"]]

def fnAPI(webRequestDelegate, startAt = 0) -> pd.DataFrame:
    def flatten_reduce_lambda(frm):
        try:
            return list(reduce(lambda x, y: x + y, frm, []))         
        except:
            return list(reduce(lambda x, y: x + y, [frm], [])) 
    def innerGetResults(webRequestDelegate, startAt = 0):
        results = webRequestDelegate(startAt)
        if isinstance(results, dict) and "total" in results and "maxResults" in results:
            if startAt + results["maxResults"] < results["total"]:
                return [results] + innerGetResults(webRequestDelegate, startAt + results["maxResults"])
            else:
                return [results]
        else:
            return [results]
    Source = flatten_reduce_lambda(innerGetResults(webRequestDelegate, startAt))
    df = pd.DataFrame(Source)
    return df


In [5]:
#! Jira Issues Capture

from IPython.display import display

ExpectedIssueTypes = [ "Task" ]
JQL = "issuetype in (" + ','.join(ExpectedIssueTypes) + ") and category in (" + ','.join(ValidProjectCategories) + ") ORDER BY updatedDate DESC"

fields = fnGetIssueTypeFields(ExpectedIssueTypes)

goldenDF = None
globals()['goldenDF'] = None 

df = fnSearch(JQL, fields["fieldId"].values)
if df is None:	
	display("No results")
else:
	df = df.drop(["expand", "startAt", "maxResults", "total"], axis=1)
	df = df.explode("issues")
	df = df.expand("issues", [], None, False)
	df = df.drop(["expand", "self"], axis=1)
	df = df.expand("fields", [], None, False)		

	df = df.rename( columns=dict( zip ( fields.fieldId, fields.name )) )	

	df = df.convert_dtypes().infer_objects().reset_index(drop=True)
	df["Id"] = df["Id"].astype('Int64')

	goldenDF:pd.DataFrame = df.copy(deep = True)
	globals()['goldenDF'] = goldenDF # make this globally available 

if 'goldenDF' not in globals() or goldenDF is None: 
	display("Base data frame not loaded") 
else:	
	display(goldenDF.dtypes)
	display(goldenDF.head())

Id                              Int64
Key                    string[python]
Issue Type                     object
Acceptance Criteria    string[python]
Risk Probability               object
Component/s                    object
Change is                      object
Sprint                         object
Requirement Status             object
Epic Link              string[python]
Description            string[python]
Project                        object
T-Shirt Size                   object
Fix Version/s                  object
Resolution                     object
Attachment                     object
Resolution Date        string[python]
Summary                string[python]
Last Viewed                    object
Delivery Teams                 object
Created                string[python]
Reporter                       object
Priority                       object
Labels                         object
Story Points                  Float64
Linked Issues                  object
Assignee    

,Id,Key,Issue Type,Acceptance Criteria,Risk Probability,Component/s,Change is,Sprint,Requirement Status,Epic Link,...,Delivery Teams,Created,Reporter,Priority,Labels,Story Points,Linked Issues,Assignee,Updated,Status
0,677349,TAP-705,{'self': 'https://jira.budgetdirect.com.au/res...,<NA>,{'self': 'https://jira.budgetdirect.com.au/res...,[],{'self': 'https://jira.budgetdirect.com.au/res...,[com.atlassian.greenhopper.service.sprint.Spri...,[],TAP-162,...,None,2023-07-04T07:33:49.000+1000,{'self': 'https://jira.budgetdirect.com.au/res...,{'self': 'https://jira.budgetdirect.com.au/res...,[],<NA>,[],{'self': 'https://jira.budgetdirect.com.au/res...,2023-07-28T09:18:25.000+1000,{'self': 'https://jira.budgetdirect.com.au/res...
1,678106,PPI-1880,{'self': 'https://jira.budgetdirect.com.au/res...,<NA>,{'self': 'https://jira.budgetdirect.com.au/res...,[{'self': 'https://jira.budgetdirect.com.au/re...,{'self': 'https://jira.budgetdirect.com.au/res...,[com.atlassian.greenhopper.service.sprint.Spri...,[],PPI-1452,...,None,2023-07-06T09:32:03.000+1000,{'self': 'https://jira.budgetdirect.com.au/res...,{'self': 'https://jira.budgetdirect.com.au/res...,[],5.0,"[{'id': '590510', 'self': 'https://jira.budget...",{'self': 'https://jira.budgetdirect.com.au/res...,2023-07-28T09:12:06.000+1000,{'self': 'https://jira.budgetdirect.com.au/res...
2,677875,PPI-1878,{'self': 'https://jira.budgetdirect.com.au/res...,<NA>,{'self': 'https://jira.budgetdirect.com.au/res...,[{'self': 'https://jira.budgetdirect.com.au/re...,{'self': 'https://jira.budgetdirect.com.au/res...,None,[],PPI-1452,...,None,2023-07-05T12:42:34.000+1000,{'self': 'https://jira.budgetdirect.com.au/res...,{'self': 'https://jira.budgetdirect.com.au/res...,[],3.0,"[{'id': '589272', 'self': 'https://jira.budget...",None,2023-07-28T09:06:42.000+1000,{'self': 'https://jira.budgetdirect.com.au/res...
3,677887,PPI-1879,{'self': 'https://jira.budgetdirect.com.au/res...,<NA>,{'self': 'https://jira.budgetdirect.com.au/res...,[{'self': 'https://jira.budgetdirect.com.au/re...,{'self': 'https://jira.budgetdirect.com.au/res...,None,[],PPI-1452,...,None,2023-07-05T13:18:50.000+1000,{'self': 'https://jira.budgetdirect.com.au/res...,{'self': 'https://jira.budgetdirect.com.au/res...,[],3.0,"[{'id': '589273', 'self': 'https://jira.budget...",None,2023-07-28T09:01:44.000+1000,{'self': 'https://jira.budgetdirect.com.au/res...
4,673985,TAP-650,{'self': 'https://jira.budgetdirect.com.au/res...,<NA>,{'self': 'https://jira.budgetdirect.com.au/res...,[],{'self': 'https://jira.budgetdirect.com.au/res...,[com.atlassian.greenhopper.service.sprint.Spri...,[],TAP-162,...,None,2023-06-20T07:53:07.000+1000,{'self': 'https://jira.budgetdirect.com.au/res...,{'self': 'https://jira.budgetdirect.com.au/res...,[],<NA>,"[{'id': '583151', 'self': 'https://jira.budget...",{'self': 'https://jira.budgetdirect.com.au/res...,2023-07-28T04:29:01.000+1000,{'self': 'https://jira.budgetdirect.com.au/res...


In [6]:
#! Issues

from IPython.display import display

if 'goldenDF' not in globals() or goldenDF is None: 
   display("Base data frame not loaded") 
else:
   df:pd.DataFrame = goldenDF.copy(deep = True)
   if df is None or len(df.index) == 0:	
      display("No results")
   else:
      df = df.drop(["Component/s", "Attachment", "Linked Issues", "Sprint", "Fix Version/s", "Labels"], axis=1)
      df = df.expand("Issue Type", ["id", "name"])
      df = df.expand("Status", ["id", "name", "statusCategory"]).sentence_case_columns()
      df = df.expand("Status Category", ["id", "name"]) 
      df = df.expand("Project", ["id", "name", "projectCategory"]).sentence_case_columns()
      df = df.expand("Project Category", ["id", "name"])
      df = df.expand("Reporter", ["key", "displayName"]) #df = df.expand("Reporter", ["accountId", "displayName"]) 
      df = df.expand("Assignee", ["key", "displayName"]) #df = df.expand("Assignee", ["accountId", "displayName"])
      df = df.expand("Priority", ["id", "name"])
      df = df.expand("Resolution", ["id", "name"])
      #df = df.expand("Parent", ["id", "key"])     
      #df = df.expand("Account", ["id", "value"])   
      #df = df.expand("Tempo Customer", ["id", "value"])   

      df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
      df["Story Points"] = df["Story Points"].astype('float')
      
      display(df.dtypes)
      display(df.head())

Id                                Int64
Key                      string[python]
Acceptance Criteria      string[python]
Risk Probability                 object
Change Is                        object
Requirement Status               object
Epic Link                string[python]
Description              string[python]
T- Shirt Size                    object
Resolution Date          string[python]
Summary                  string[python]
Last Viewed                      object
Delivery Teams                   object
Created                  string[python]
Story Points                    float64
Updated                  string[python]
Issue Type Id            string[python]
Issue Type Name          string[python]
Status Id                string[python]
Status Name              string[python]
Status Category Id                Int64
Status Category Name     string[python]
Project Id               string[python]
Project Name             string[python]
Project Category Id      string[python]


,Id,Key,Acceptance Criteria,Risk Probability,Change Is,Requirement Status,Epic Link,Description,T- Shirt Size,Resolution Date,...,Project Category Id,Project Category Name,Reporter Key,Reporter Display Name,Assignee Key,Assignee Display Name,Priority Id,Priority Name,Resolution Id,Resolution Name
0,677349,TAP-705,<NA>,{'self': 'https://jira.budgetdirect.com.au/res...,{'self': 'https://jira.budgetdirect.com.au/res...,[],TAP-162,* Difficult to execute * Requires a lot re-r...,None,2023-07-28T09:18:25.000+1000,...,10930,Portfolio,JIRAUSER32846,Shaun Loh,JIRAUSER32846,Shaun Loh,2,Major,10000,Done
1,678106,PPI-1880,<NA>,{'self': 'https://jira.budgetdirect.com.au/res...,{'self': 'https://jira.budgetdirect.com.au/res...,[],PPI-1452,We need to coordinate some after-hours technic...,None,2023-07-28T09:12:06.000+1000,...,10930,Portfolio,JIRAUSER32701,Peter Day,JIRAUSER31636,Paulo Ogliani,2,Major,10000,Done
2,677875,PPI-1878,<NA>,{'self': 'https://jira.budgetdirect.com.au/res...,{'self': 'https://jira.budgetdirect.com.au/res...,[],PPI-1452,For {color:#172b4d}*Jira Prod - [https://jira....,None,2023-07-28T09:06:42.000+1000,...,10930,Portfolio,JIRAUSER32701,Peter Day,<NA>,<NA>,2,Major,10000,Done
3,677887,PPI-1879,<NA>,{'self': 'https://jira.budgetdirect.com.au/res...,{'self': 'https://jira.budgetdirect.com.au/res...,[],PPI-1452,JCMA introduced a new preflight in 1.7.3 that ...,None,2023-07-28T09:01:44.000+1000,...,10930,Portfolio,JIRAUSER32701,Peter Day,<NA>,<NA>,2,Major,10000,Done
4,673985,TAP-650,<NA>,{'self': 'https://jira.budgetdirect.com.au/res...,{'self': 'https://jira.budgetdirect.com.au/res...,[],TAP-162,<NA>,None,<NA>,...,10930,Portfolio,JIRAUSER32846,Shaun Loh,JIRAUSER32846,Shaun Loh,4,Minor,<NA>,<NA>


In [7]:
#! Components

from IPython.display import display

if 'goldenDF' not in globals() or goldenDF is None: 
    display("Base data frame not loaded") 
else:
    df:pd.DataFrame = goldenDF.copy(deep = True)
    df = df[["Id", "Key", "Component/s"]]
    df = df[df["Component/s"].map(lambda d: len(d)) > 0]

    if df is None or len(df.index) == 0:	
        display("No results")
    else:
        df = df.explode("Component/s")
        df = df.expand("Component/s", ["id", "name"])

        df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)

        display(df.dtypes)
        display(df.head())

Id                           Int64
Key                 string[python]
Component/s Id      string[python]
Component/s Name    string[python]
dtype: object

,Id,Key,Component/s Id,Component/s Name
0,678106,PPI-1880,26922,Clouding_Atlassian
1,677875,PPI-1878,26922,Clouding_Atlassian
2,677887,PPI-1879,26922,Clouding_Atlassian
3,653235,PPI-1724,26922,Clouding_Atlassian
4,653145,PPI-1722,26922,Clouding_Atlassian


In [8]:
#! Linked Issues

from IPython.display import display

if 'goldenDF' not in globals() or goldenDF is None: 
    display("Base data frame not loaded") 
else:
    df:pd.DataFrame = goldenDF.copy(deep = True)
    df = df[["Id", "Key", "Linked Issues"]]
    df = df[df["Linked Issues"].map(lambda d: len(d)) > 0]

    if df is None or len(df.index) == 0:	
        display("No results")
    else:
        df = df.explode("Linked Issues")
        df = df.expand("Linked Issues", ["id", "type", "inwardIssue", "outwardIssue"])

        df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)

        display(df.dtypes)
        display(df.head())

Id                                      Int64
Key                            string[python]
Linked Issues Id               string[python]
Linked Issues Type                     object
Linked Issues Inward Issue             object
Linked Issues Outward Issue            object
dtype: object

,Id,Key,Linked Issues Id,Linked Issues Type,Linked Issues Inward Issue,Linked Issues Outward Issue
0,678106,PPI-1880,590510,"{'id': '10650', 'name': 'Related Issues', 'inw...","{'id': '679030', 'key': 'CSI-1820', 'self': 'h...",NaN
1,678106,PPI-1880,590512,"{'id': '10650', 'name': 'Related Issues', 'inw...","{'id': '679032', 'key': 'CSI-1821', 'self': 'h...",NaN
2,678106,PPI-1880,589272,"{'id': '10650', 'name': 'Related Issues', 'inw...","{'id': '677875', 'key': 'PPI-1878', 'self': 'h...",NaN
3,678106,PPI-1880,589273,"{'id': '10650', 'name': 'Related Issues', 'inw...","{'id': '677887', 'key': 'PPI-1879', 'self': 'h...",NaN
4,677875,PPI-1878,589272,"{'id': '10650', 'name': 'Related Issues', 'inw...",NaN,"{'id': '678106', 'key': 'PPI-1880', 'self': 'h..."


In [9]:
#! Sprints

from IPython.display import display
from dateutil import parser
from pandas.core.dtypes.dtypes import DatetimeTZDtype

if 'goldenDF' not in globals() or goldenDF is None: 
    display("Base data frame not loaded") 
else:
    df:pd.DataFrame = goldenDF.copy(deep = True)
    df = df[["Id", "Key", "Sprint"]]
    df = df[~df["Sprint"].isna()]

    if df is None or len(df.index) == 0:	
        display("No results")
    else:		
        df = df.explode("Sprint")
        df = df.expand("Sprint")

        if "Sprint Start Date" not in df.columns.values:
            # Server instance not cloud 
            def fixSprint(s):
                try:
                    s = (s.split('[', 1)[1])[:-1]
                    res = []
                    for sub in s.split(','):
                        if '=' in sub:
                            res.append(map(str.strip, sub.split('=', 1)))
                    return dict(res)
                except: 
                    return None
            df["Sprint"] = df["Sprint"].apply (fixSprint)
            df = df.expand("Sprint")

            def fixDate(s):
                try:
                    return parser.isoparse(s)
                except:
                    return None
            df["Sprint Start Date"] = df["Sprint Start Date"].apply(fixDate)
            df["Sprint End Date"] = df["Sprint End Date"].apply(fixDate)
            df["Sprint Complete Date"] = df["Sprint Complete Date"].apply(fixDate)
            
        df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)

        df["Sprint Start Date"] = df["Sprint Start Date"].astype(DatetimeTZDtype("ns", "+10:00"))
        df["Sprint End Date"] = df["Sprint End Date"].astype(DatetimeTZDtype("ns", "+10:00"))
        df["Sprint Complete Date"] = df["Sprint Complete Date"].astype(DatetimeTZDtype("ns", "+10:00"))
        
        display(df.dtypes)
        display(df.head())

Id                                            Int64
Key                                  string[python]
Sprint Id                            string[python]
Sprint Rapid View Id                 string[python]
Sprint State                         string[python]
Sprint Name                          string[python]
Sprint Start Date         datetime64[ns, UTC+10:00]
Sprint End Date           datetime64[ns, UTC+10:00]
Sprint Complete Date      datetime64[ns, UTC+10:00]
Sprint Activated Date                string[python]
Sprint Sequence                      string[python]
Sprint Goal                          string[python]
Sprint Auto Start Stop               string[python]
dtype: object

,Id,Key,Sprint Id,Sprint Rapid View Id,Sprint State,Sprint Name,Sprint Start Date,Sprint End Date,Sprint Complete Date,Sprint Activated Date,Sprint Sequence,Sprint Goal,Sprint Auto Start Stop
0,677349,TAP-705,6207,1443,CLOSED,TAP: Sprint 12,2023-07-04 09:00:00+10:00,2023-07-17 06:30:00+10:00,2023-07-16 22:32:56.267000+10:00,2023-07-04T09:06:36.094+10:00,6207,<null>,false
1,677349,TAP-705,6275,1443,ACTIVE,TAP: Sprint 13,2023-07-17 12:00:00+10:00,2023-07-31 09:00:00+10:00,NaT,2023-07-18T08:23:16.629+10:00,6275,,false
2,678106,PPI-1880,6345,1116,ACTIVE,PPI-FY24Q1-S3,2023-07-24 11:59:00+10:00,2023-08-04 17:00:00+10:00,NaT,2023-07-24T12:09:15.674+10:00,6345,GM: Deliver ELT Offsite Workshop Testing Goal...,false
3,673985,TAP-650,6120,1443,CLOSED,TAP: Sprint 11,2023-06-20 08:36:00+10:00,2023-07-03 06:00:00+10:00,2023-07-03 20:54:06.047000+10:00,2023-06-20T08:37:10.323+10:00,6120,,false
4,673985,TAP-650,6207,1443,CLOSED,TAP: Sprint 12,2023-07-04 09:00:00+10:00,2023-07-17 06:30:00+10:00,2023-07-16 22:32:56.267000+10:00,2023-07-04T09:06:36.094+10:00,6207,<null>,false


In [10]:
#! Attachments

from IPython.display import display
from pandas.core.dtypes.dtypes import DatetimeTZDtype

if 'goldenDF' not in globals() or goldenDF is None: 
    display("Base data frame not loaded") 
else:
    df:pd.DataFrame = goldenDF.copy(deep = True)
    df = df[["Id", "Key", "Attachment"]]
    df = df[df["Attachment"].map(lambda d: len(d)) > 0]

    if df is None or len(df.index) == 0:	
        display("No results")
    else:
        df = df.explode("Attachment")
        df = df.expand("Attachment", ["id", "filename", "created", "mimeType", "size", "thumbnail", "content", "author"])
        df = df.expand("Attachment Author", ["key"])

        df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
        df["Attachment Id"] = df["Attachment Id"].astype('Int64')
        df["Attachment Size"] = df["Attachment Size"].astype('float64')
        df["Attachment Created"] = df["Attachment Created"].astype(DatetimeTZDtype("ns", "+10:00"))

        display(df.dtypes)
        display(df.head())

Id                                           Int64
Key                                 string[python]
Attachment Id                                Int64
Attachment Filename                 string[python]
Attachment Created       datetime64[ns, UTC+10:00]
Attachment Mime Type                string[python]
Attachment Size                            float64
Attachment Thumbnail                string[python]
Attachment Content                  string[python]
Attachment Author Key               string[python]
dtype: object

,Id,Key,Attachment Id,Attachment Filename,Attachment Created,Attachment Mime Type,Attachment Size,Attachment Thumbnail,Attachment Content,Attachment Author Key
0,653073,PPI-1720,562909,AGP-3267_Scenario_Security_ADMIN_JIRA Cloud_15...,2023-03-15 13:25:14.638000+10:00,application/vnd.openxmlformats-officedocument....,54246.0,<NA>,https://jira.budgetdirect.com.au/secure/attach...,JIRAUSER33364
1,653068,PPI-1719,562904,AGP-3267_Scenario_Security_ADMIN_Confluence_15...,2023-03-15 13:14:55.176000+10:00,application/vnd.openxmlformats-officedocument....,42421.0,<NA>,https://jira.budgetdirect.com.au/secure/attach...,JIRAUSER33364
2,679942,TAP-729,592883,image-2023-07-12-11-31-03-116.png,2023-07-12 11:31:03.260000+10:00,image/png,81331.0,https://jira.budgetdirect.com.au/secure/thumbn...,https://jira.budgetdirect.com.au/secure/attach...,JIRAUSER32583
3,681717,PPI-1898,594463,Integrity01.png,2023-07-18 12:08:49.072000+10:00,image/png,31162.0,https://jira.budgetdirect.com.au/secure/thumbn...,https://jira.budgetdirect.com.au/secure/attach...,JIRAUSER32701
4,681717,PPI-1898,594464,Integrity02.png,2023-07-18 12:08:56.400000+10:00,image/png,51690.0,https://jira.budgetdirect.com.au/secure/thumbn...,https://jira.budgetdirect.com.au/secure/attach...,JIRAUSER32701


In [11]:
#! Versions

from IPython.display import display

if 'goldenDF' not in globals() or goldenDF is None: 
    display("Base data frame not loaded") 
else:
    df:pd.DataFrame = goldenDF.copy(deep = True)
    df = df[["Id", "Key", "Fix Version/s"]]
    df = df[df["Fix Version/s"].map(lambda d: len(d)) > 0]

    if df is None or len(df.index) == 0:	
        display("No results")
    else:
        df = df.explode("Fix Version/s")
        df = df.expand("Fix Version/s", ["id", "name"])

        df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
        df["Fix Version/s Id"] = df["Fix Version/s Id"].astype('Int64')

        display(df.dtypes)
        display(df.head())

Id                             Int64
Key                   string[python]
Fix Version/s Id               Int64
Fix Version/s Name    string[python]
dtype: object

,Id,Key,Fix Version/s Id,Fix Version/s Name
0,574677,PPI-713,22512,Portfolio
1,575501,PPI-723,22512,Portfolio
2,640741,FLOW-386,22629,FY22Q3
3,640739,FLOW-385,22629,FY22Q3
4,575593,PPI-728,22512,Portfolio
